In [1]:
#!/usr/bin/env python3

import pandas as pd
from pathlib import Path
import acts
import acts.examples
from acts.examples import DigitizationCoordinatesConverter, readDigiConfigFromJson

def convert_digitized_hits_csv(csv_file: Path, digi_config_file: Path):
    print(f"[INFO] Converting digitized CSV hits to global coordinates")

    # Check input files
    assert csv_file.exists(), f"Input CSV does not exist: {csv_file}"
    assert digi_config_file.exists(), f"Digitization config not found: {digi_config_file}"

    # Load measurement data
    df = pd.read_csv(csv_file)

    # Build ACTS geometry (GenericDetector)
    detector = acts.examples.GenericDetector()
    tracking_geometry = detector.trackingGeometry()

    # Build Digitization config for the coordinate converter
    digi_config = acts.examples.DigitizationAlgorithm.Config(
        digitizationConfigs=readDigiConfigFromJson(str(digi_config_file.resolve())),
        surfaceByIdentifier=tracking_geometry.geoIdSurfaceMap()
    )
    converter = DigitizationCoordinatesConverter(digi_config)

    # Convert all local coordinates to global
    # global_coords = []
    # for row in df.itertuples(index=False):
    #     try:
    #         g = converter.localToGlobal(int(row.geometry_id), float(row.local0), float(row.local1))
    #     except RuntimeError:
    #         g = (float("nan"), float("nan"), float("nan"))
    #     global_coords.append(g)
    
    # x, y, z = zip(*global_coords)
    # df["global_x"] = x
    # df["global_y"] = y
    # df["global_z"] = z
    global_coords = []
    volumes = []
    layers = []
    modules = []
    
    for row in df.itertuples(index=False):
        try:
            geo_id = acts.GeometryIdentifier(int(row.geometry_id))
            volumes.append(geo_id.volume)
            layers.append(geo_id.layer)
            modules.append(geo_id.sensitive)
    
            g = converter.localToGlobal(int(row.geometry_id), float(row.local0), float(row.local1))
        except RuntimeError:
            g = (float("nan"), float("nan"), float("nan"))
            volumes.append(-1)
            layers.append(-1)
            modules.append(-1)
    
        global_coords.append(g)

    x, y, z = zip(*global_coords)
    df["global_x"] = x
    df["global_y"] = y
    df["global_z"] = z
    df["volume"] = volumes
    df["layer"] = layers
    df["module"] = modules



    # Write to CSV
    out_csv = csv_file.with_name("digitized_hits_global_xyz.csv")
    df.to_csv(out_csv, index=False)
    print(f"[✓] Saved: {out_csv}")

In [2]:
csv_input = Path("ttbar_H_production/csv/signal/digitization/event000000000-measurements.csv")
digi_config = Path("acts-41.1.0/Examples/Configs/generic-digi-geometric-config.json")

convert_digitized_hits_csv(csv_input, digi_config)

[INFO] Converting digitized CSV hits to global coordinates
15:08:09    GenericDetec   INFO      Building tracking geometry
15:08:09    GenericDetec   INFO      Building tracking geometry for Generic Detector in Gen1 mode
15:08:09    GenericDetec   INFO      Tracking geometry built
[✓] Saved: ttbar_H_production/csv/signal/digitization/digitized_hits_global_xyz.csv


In [6]:
df = pd.read_csv(csv_input)
print(df.columns)


Index(['measurement_id', 'geometry_id', 'local_key', 'local0', 'local1', 'phi',
       'theta', 'time', 'var_local0', 'var_local1', 'var_phi', 'var_theta',
       'var_time'],
      dtype='object')
